# Comparing the data using altair

In [1]:
# Import Libraries and Dependencies 
import pandas as pd
import sqlite3
import sys
import requests
from ipywidgets import interact, widgets, fixed
from IPython.display import display
import altair as alt


In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('../database/db.sqlite')

# Query to select data from the nutrition table
query = "SELECT * FROM nutrition;"

# Read data into a DataFrame
menu_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

In [3]:
# preview the df
menu_df.head()

,id,Restaurant,Item,item_with_chain,Calories,Calories From Fat,Total Fat,Saturated Fat,Trans Fat,Cholesterol,Sodium,Total Carbohydrates,Fiber,Sugar,Protein
0,1,Mcdonalds,Artisan Grilled Chicken Sandwich,Artisan Grilled Chicken Sandwich (Mcdonalds),380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0
1,2,Mcdonalds,Single Bacon Smokehouse Burger,Single Bacon Smokehouse Burger (Mcdonalds),840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0
2,3,Mcdonalds,Double Bacon Smokehouse Burger,Double Bacon Smokehouse Burger (Mcdonalds),1130.0,600.0,67.0,27.0,3.0,220.0,1920.0,63.0,3.0,18.0,70.0
3,4,Mcdonalds,Grilled Bacon Smokehouse Chicken Sandwich,Grilled Bacon Smokehouse Chicken Sandwich (Mcd...,750.0,280.0,31.0,10.0,0.5,155.0,1940.0,62.0,2.0,18.0,55.0
4,5,Mcdonalds,Crispy Bacon Smokehouse Chicken Sandwich,Crispy Bacon Smokehouse Chicken Sandwich (Mcdo...,920.0,410.0,45.0,12.0,0.5,120.0,1980.0,81.0,4.0,18.0,46.0


In [15]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   513 non-null    int64  
 1   Restaurant           513 non-null    object 
 2   Item                 513 non-null    object 
 3   item_with_chain      513 non-null    object 
 4   Calories             513 non-null    float64
 5   Calories From Fat    513 non-null    float64
 6   Total Fat            513 non-null    float64
 7   Saturated Fat        513 non-null    float64
 8   Trans Fat            513 non-null    float64
 9   Cholesterol          513 non-null    float64
 10  Sodium               513 non-null    float64
 11  Total Carbohydrates  513 non-null    float64
 12  Fiber                513 non-null    float64
 13  Sugar                513 non-null    float64
 14  Protein              513 non-null    float64
dtypes: float64(11), int64(1), object(3)
memo

In [4]:
# Using altair to compare two items

# Enable notebook rendering for Altair
# https://altair-viz.github.io/user_guide/display_frontends.html
# https://stackoverflow.com/questions/59705914/not-able-to-display-altair-charts-in-jupyter-notebook
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text="Comparing the Nutrients of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)



# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [16]:


# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_column):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()

    # Filter only the selected column for both items
    item_data1 = item_data1[[selected_column]]
    item_data2 = item_data2[[selected_column]]

    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text=f"Comparing {selected_column} of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)

# Create dropdown widgets for items and columns
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])
column_dropdown = widgets.Dropdown(options=menu_df.columns[3:].tolist(), description='Select Column:', value='Calories')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_column=column_dropdown)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_column)>

In [17]:
# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Assume 'menu_df' is your DataFrame containing the nutrition data
# You can load your data using: menu_df = pd.read_csv('your_data.csv')

# Define a function to display information for the selected restaurants
def display_restaurants_info(selected_restaurant1, selected_restaurant2, selected_column):
    restaurant_data1 = menu_df[menu_df['Restaurant'] == selected_restaurant1]
    restaurant_data2 = menu_df[menu_df['Restaurant'] == selected_restaurant2]

    # Calculate mean values for the selected column for both restaurants
    mean_values1 = restaurant_data1[selected_column].mean()
    mean_values2 = restaurant_data2[selected_column].mean()

    # Create a DataFrame with mean values
    comparison_df = pd.DataFrame({
        selected_restaurant1: [mean_values1],
        selected_restaurant2: [mean_values2]
    })

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Restaurant', value_name='Mean Value')

    # Create the chart
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Mean Value:Q', title=' '),
        color=alt.Color('Restaurant:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Restaurant:N', alt.Tooltip('Mean Value:Q', title='Mean Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Restaurant:N',
        title=alt.TitleParams(text=f"Comparing Mean {selected_column} of the Two Selected Restaurants", offset=10, orient='top')
    )

    display(chart)

# Create dropdown widgets for restaurants and columns
restaurant_dropdown1 = widgets.Dropdown(options=menu_df['Restaurant'].unique(), description='Select Restaurant 1:')
restaurant_dropdown2 = widgets.Dropdown(options=menu_df['Restaurant'].unique(), description='Select Restaurant 2:', value=menu_df['Restaurant'].unique()[1])
column_dropdown = widgets.Dropdown(options=menu_df.columns[3:].tolist(), description='Select Column:', value='Calories')

# Connect the dropdown widgets to the display_restaurants_info function
interact(display_restaurants_info, selected_restaurant1=restaurant_dropdown1, selected_restaurant2=restaurant_dropdown2, selected_column=column_dropdown)

interactive(children=(Dropdown(description='Select Restaurant 1:', options=('Mcdonalds', 'Chick Fil-A', 'Sonic…

<function __main__.display_restaurants_info(selected_restaurant1, selected_restaurant2, selected_column)>